Downloading and re-arranging the dataset was difficult, and I found it online from Cornell directly all cleaned and viable. Using the link to the [Main Site](https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html) which then takes you to the [library download](https://convokit.cornell.edu/documentation/movie.html)

In [1]:
pip install convokit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.2/183.2 kB 1.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 3.5 MB/s eta 0:00:00
  Created wheel for convokit: filename=convokit-3.0.0-py3-none-any.whl size=216707 sha256=06e9e28562e1f8bfb5f60e0fa239a2fb938e217f0ce78d6d54dfa44095b9d8dd
  Stored in directory: /root/.cache/pip/wheels/c4/89/8c/2677fdb888588b6f93cb6ac86bdfb020f1f1c33e0d5525b231
 

In [2]:
!pip install tabulate

In [3]:
pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.1 MB/s eta 0:00:00


In [4]:
pip install uvicorn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.6 MB/s eta 0:00:00


In [5]:
pip install fastapi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.0 MB/s eta 0:00:00


In [6]:
#Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import torch
import uvicorn
import math
import optuna
from sklearn.model_selection import train_test_split
from tabulate import tabulate
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, EarlyStoppingCallback
from torch.utils.data import Dataset, DataLoader
from multiprocessing import Pool, cpu_count
from sklearn.metrics import accuracy_score, recall_score, f1_score
from nltk.corpus import wordnet
from fastapi import FastAPI
from google.colab import files

 - Using the corpus import from convokit as part of Cornell's libraries, we first download the data

In [7]:
from convokit import Corpus, download
corpus = Corpus(filename=download("movie-corpus"))

No configuration file found at /root/.convokit/config.yml; writing with contents: 
# Default Backend Parameters
db_host: localhost:27017
data_directory: ~/.convokit/saved-corpora
default_backend: mem


In [8]:
corpus.print_summary_stats()

Number of Speakers: 9035
Number of Utterances: 304713
Number of Conversations: 83097


 - Check metadata

In [9]:
print(corpus.meta)

ConvoKitMeta({'url': {'m0': 'http://www.dailyscript.com/scripts/10Things.html', 'm1': 'http://www.hundland.org/scripts/1492-ConquestOfParadise.txt', 'm2': 'http://www.dailyscript.com/scripts/15minutes.html', 'm3': 'http://www.scifiscripts.com/scripts/2001.txt', 'm4': 'http://www.awesomefilm.com/script/48hours.txt', 'm5': 'http://www.scifiscripts.com/scripts/5thelement.txt', 'm6': 'http://www.dailyscript.com/scripts/eight-millimeter.html', 'm7': 'http://www.hundland.org/scripts/A-Nightmare-on-Elm-Street-4.txt', 'm8': 'http://www.hundland.org/scripts/A-Nightmare-on-Elm-Street-5.txt', 'm9': 'http://leonscripts.tripod.com/scripts/ATOMICSUB.htm', 'm10': 'http://www.dailyscript.com/scripts/Affliction.txt', 'm11': 'http://www.dailyscript.com/scripts/AirForceOne_TXT.html', 'm12': 'http://www.dailyscript.com/scripts/Airplane2_script.htm', 'm13': 'http://www.dailyscript.com/scripts/Airplane_script.htm', 'm14': 'http://www.dailyscript.com/scripts/Alien_Nation_Bannon_Cameron_October_1987.html', 'm

 - Print out metadata to see all features and categories we can work with

In [10]:
for conversation in list(corpus.iter_conversations())[:5]:
    print(f"Conversation ID: {conversation.id}")
    print("Metadata:", conversation.meta)
    print("-" * 50)

Conversation ID: L1044
Metadata: ConvoKitMeta({'movie_idx': 'm0', 'movie_name': '10 things i hate about you', 'release_year': '1999', 'rating': '6.90', 'votes': '62847', 'genre': "['comedy', 'romance']"})
--------------------------------------------------
Conversation ID: L984
Metadata: ConvoKitMeta({'movie_idx': 'm0', 'movie_name': '10 things i hate about you', 'release_year': '1999', 'rating': '6.90', 'votes': '62847', 'genre': "['comedy', 'romance']"})
--------------------------------------------------
Conversation ID: L924
Metadata: ConvoKitMeta({'movie_idx': 'm0', 'movie_name': '10 things i hate about you', 'release_year': '1999', 'rating': '6.90', 'votes': '62847', 'genre': "['comedy', 'romance']"})
--------------------------------------------------
Conversation ID: L870
Metadata: ConvoKitMeta({'movie_idx': 'm0', 'movie_name': '10 things i hate about you', 'release_year': '1999', 'rating': '6.90', 'votes': '62847', 'genre': "['comedy', 'romance']"})
------------------------------

 - From the above, we have all features related to the dataset as presented on kaggle. The issue now is to merge all metadata into a single, unified dataframe for data manipulation and training.

##1) Extracting all utterances into a DF

In [11]:
#Take all "utterences" from metadata and add to list
#Want to maintain utterance_id as a reference point for other features
utterances_data = []
for utterance in corpus.iter_utterances():
    utterances_data.append({
        'utterance_id': utterance.id,
        'text': utterance.text,
        'speaker_id': utterance.speaker.id,
        'conversation_id': utterance.conversation_id,
        'reply_to': utterance.reply_to
    })

utterances_df = pd.DataFrame(utterances_data)

In [12]:
#Going to drop duplicates and rows with missing texts ince it is most important piece of information
utterances_df.drop_duplicates(subset=['utterance_id'], inplace=True)
utterances_df.dropna(subset=['text'], inplace=True)
utterances_df.sort_values(by=['conversation_id', 'utterance_id'], inplace=True)
utterances_df.reset_index(drop=True, inplace=True)

##2) Extract Movie Metadata

In [13]:
#Initialize our lists for some metadata features
movie_ids = []
release_years = []
ratings = []
votes = []
genres = []

#Iterate over the conversations in corpus to get metadata
for conversation in corpus.iter_conversations():
    movie_ids.append(conversation.id)
    release_years.append(conversation.meta.get('release_year', None))
    ratings.append(conversation.meta.get('rating', None))
    votes.append(conversation.meta.get('votes', None))
    genres.append(conversation.meta.get('genre', None))

#DataFRame created for metadata storage
movie_metadata_df = pd.DataFrame({
    'movie_id': movie_ids,
    'year': release_years,
    'rating': ratings,
    'votes': votes,
    'genres': genres
})

In [14]:
movie_metadata_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83097 entries, 0 to 83096
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  83097 non-null  object
 1   year      83097 non-null  object
 2   rating    83097 non-null  object
 3   votes     83097 non-null  object
 4   genres    83097 non-null  object
dtypes: object(5)
memory usage: 3.2+ MB


In [15]:
#some columns are numerical, going to shift to different type.
movie_metadata_df['year'] = pd.to_numeric(movie_metadata_df['year'], errors='coerce')
movie_metadata_df['rating'] = pd.to_numeric(movie_metadata_df['rating'], errors='coerce')
movie_metadata_df['votes'] = pd.to_numeric(movie_metadata_df['votes'], errors='coerce')

#Genre's will be of list type for simplification down the line
movie_metadata_df['genres'] = movie_metadata_df['genres'].apply(lambda x: eval(x) if pd.notna(x) else [])


##3) Character Metadata

In [16]:
#More lists for other metadata features
speaker_ids = []
character_names = []
genders = []
credit_positions = []

#Extract metadata from speakers
for speaker in corpus.iter_speakers():
    speaker_ids.append(speaker.id)
    #Name may be missing, so we will include "Unknown" classifier
    character_names.append(speaker.meta.get('character_name', "Unknown"))
    genders.append(speaker.meta.get('gender', None))
    credit_positions.append(speaker.meta.get('credit_pos', None))

#DataFrame created for metadata storage specific to character data
character_metadata_df = pd.DataFrame({
    'speaker_id': speaker_ids,
    'character_name': character_names,
    'gender': genders,
    'credit_position': credit_positions
})

In [17]:
#Created dictionary to map Speaker ID to character name
character_dict = dict(zip(character_metadata_df['speaker_id'], character_metadata_df['character_name']))


##4) Merging Metadata

 - As seen above, we have a few dataframes pulling metadata from the library, now we need to merge into a single dataframe.

In [18]:
#Merge movie metadata into utterances_df based on 'conversation_id'
utterances_with_metadata = pd.merge(utterances_df, movie_metadata_df, left_on='conversation_id', right_on='movie_id', how='left')

#Merge character metadata into utterances_df based on 'speaker_id'
utterances_with_metadata = pd.merge(utterances_with_metadata, character_metadata_df, on='speaker_id', how='left')


##5) Context-Response Pairs by ID

 - Going to create context-response pairs with character names to add context to input/response as it may be a useful feature for our model

In [19]:
#Initializing lists to store context/response pairs and associated character names
contexts = []
responses = []
context_characters = []
response_characters = []
pair_ids = []

#Dictionary for utterences and speaker ID's
utterance_text_dict = dict(zip(utterances_with_metadata['utterance_id'], utterances_with_metadata['text']))
utterance_speaker_dict = dict(zip(utterances_with_metadata['utterance_id'], utterances_with_metadata['speaker_id']))

#Loop through the utterences, build context/response pairs
for index, row in utterances_with_metadata.iterrows():
    #Filter in rows with a valid 'reply_to' since this makes up our context
    if pd.notna(row['reply_to']):
        context_id = row['reply_to']
        response_id = row['utterance_id']

        #Grab context and response texts
        context_text = utterance_text_dict.get(context_id, "")
        response_text = row['text']

        #Grab speaker ID's for context and response
        context_speaker_id = utterance_speaker_dict.get(context_id, None)
        response_speaker_id = row['speaker_id']

        #Grab character names
        context_character = character_dict.get(context_speaker_id, "Unknown")
        response_character = character_dict.get(response_speaker_id, "Unknown")

        #Append all information to initalized lists
        contexts.append(context_text)
        responses.append(response_text)
        context_characters.append(context_character)
        response_characters.append(response_character)
        pair_ids.append(response_id)

#Create a new DataFrame for context/response pairs including character names
dialogue_pairs_with_metadata = pd.DataFrame({
    'ID': pair_ids,
    'context': contexts,
    'context_character': context_characters,
    'response': responses,
    'response_character': response_characters
})

#Merge with movie metadata extracted from earlier
dialogue_pairs_with_metadata = pd.merge(
    dialogue_pairs_with_metadata,
    utterances_with_metadata[['utterance_id', 'year', 'rating', 'votes', 'genres', 'gender', 'credit_position']],
    left_on='ID',
    right_on='utterance_id',
    how='left'
)

##6) Merging Context-Response Pairs with Metadata for complete Df

In [20]:
#Column selection for parent DataFrame
final_columns = ['ID', 'context', 'context_character', 'response', 'response_character',
                 'year', 'rating', 'votes', 'genres', 'gender', 'credit_position']
dialogue_pairs_with_metadata = dialogue_pairs_with_metadata[final_columns]

#Check and ensure that it structured properly
print(dialogue_pairs_with_metadata.head())

        ID                                            context  \
0  L100002  Then why did you go see Mr. Koehler in the fir...   
1  L100004                                           Hi, Joe.   
2  L100005                  Frank -- what are you doing here?   
3  L100006  I need to ask you a question.  What made you g...   
4  L100007                  You know I can't talk about that.   

  context_character                                           response  \
0           DULANEY                              Because he called me.   
1           DULANEY                  Frank -- what are you doing here?   
2           KOEHLER  I need to ask you a question.  What made you g...   
3           DULANEY                  You know I can't talk about that.   
4           KOEHLER  I'm not asking for names or specifics.  I just...   

  response_character    year  rating  votes                      genres  \
0             JOANNE  1993.0     4.1   6052  [drama, romance, thriller]   
1            K

 - Dataframe looks ugly here, going to format display into a table

In [21]:
print(tabulate(dialogue_pairs_with_metadata.head(), headers='keys', tablefmt='psql'))

+----+---------+--------------------------------------------------------------------------------+---------------------+-------------------------------------------------------------------------------------------------+----------------------+--------+----------+---------+----------------------------------+----------+-------------------+
|    | ID      | context                                                                        | context_character   | response                                                                                        | response_character   |   year |   rating |   votes | genres                           | gender   | credit_position   |
|----+---------+--------------------------------------------------------------------------------+---------------------+-------------------------------------------------------------------------------------------------+----------------------+--------+----------+---------+----------------------------------+----------+----------

In [22]:
dialogue_pairs_with_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221616 entries, 0 to 221615
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ID                  221616 non-null  object 
 1   context             221616 non-null  object 
 2   context_character   221616 non-null  object 
 3   response            221616 non-null  object 
 4   response_character  221616 non-null  object 
 5   year                216702 non-null  float64
 6   rating              221616 non-null  float64
 7   votes               221616 non-null  int64  
 8   genres              221616 non-null  object 
 9   gender              221616 non-null  object 
 10  credit_position     221616 non-null  object 
dtypes: float64(2), int64(1), object(8)
memory usage: 18.6+ MB


 - Info shows mostly perfect data, but still a few issues seen in displayed data, primarily that we have '?' in gender and credit position for unknowns. This is essentially a substitute for NaN and needs management


##7) Handle Missing Values and '?'

In [23]:
#Missing Data printout
print("Missing values per column:")
print(dialogue_pairs_with_metadata.isnull().sum())

#Question mark check in column values
placeholder_counts = (dialogue_pairs_with_metadata == '?').sum()
print("\nCount of '?' placeholders per column:")
print(placeholder_counts)

Missing values per column:
ID                       0
context                  0
context_character        0
response                 0
response_character       0
year                  4914
rating                   0
votes                    0
genres                   0
gender                   0
credit_position          0
dtype: int64

Count of '?' placeholders per column:
ID                        0
context                   0
context_character         0
response                  0
response_character        0
year                      0
rating                    0
votes                     0
genres                    0
gender                43082
credit_position       62395
dtype: int64


 - Features 'Year' has missing values, with 'Credit_Position'and 'Gender' having '?' values.

 - We will handle gender and credit_position first

In [24]:
#Replace '?' with 'Unknown' in gender instead of NaN
dialogue_pairs_with_metadata['gender'].replace('?', 'Unknown', inplace=True)

#For credit_position going with a placeholder value of -1 to represent unknown
dialogue_pairs_with_metadata['credit_position'].replace('?', -1, inplace=True)

<ipython-input-24-cbe33e858ac5>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dialogue_pairs_with_metadata['gender'].replace('?', 'Unknown', inplace=True)
<ipython-input-24-cbe33e858ac5>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(val

 - Handle missing values of year now

In [25]:
#Substitute missing values with median values of existing years.
median_year = dialogue_pairs_with_metadata['year'].median()
dialogue_pairs_with_metadata['year'].fillna(median_year, inplace=True)

<ipython-input-25-808aa0832217>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dialogue_pairs_with_metadata['year'].fillna(median_year, inplace=True)


 - Double check to see that they've been eliminated

In [26]:
print("Remaining '?' in `gender`:", (dialogue_pairs_with_metadata['gender'] == '?').sum())
print("Remaining '?' in `credit_position`:", (dialogue_pairs_with_metadata['credit_position'] == '?').sum())
print("Missing values in `year` after filling:", dialogue_pairs_with_metadata['year'].isnull().sum())


Remaining '?' in `gender`: 0
Remaining '?' in `credit_position`: 0
Missing values in `year` after filling: 0


 - Now we check datatypes to address any issues there

In [27]:
print(dialogue_pairs_with_metadata.dtypes)

ID                     object
context                object
context_character      object
response               object
response_character     object
year                  float64
rating                float64
votes                   int64
genres                 object
gender                 object
credit_position        object
dtype: object


 - Weird that credit_position is an object, let's change that

In [28]:
#Convert credit_position to type int
dialogue_pairs_with_metadata['credit_position'] = dialogue_pairs_with_metadata['credit_position'].astype(int)
print(dialogue_pairs_with_metadata.dtypes)

ID                     object
context                object
context_character      object
response               object
response_character     object
year                  float64
rating                float64
votes                   int64
genres                 object
gender                 object
credit_position         int64
dtype: object


 - The data is good, needs some more preprocessing (check for class imbalance, etc.) but we can proceed with model and initialization

In [29]:
#Initializing with special tokens
#Using smallest GPT model
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
tokenizer.add_special_tokens({
    'pad_token': '[PAD]',
    'bos_token': '<SOS>',
    'eos_token': '<EOS>'
})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


3

In [30]:
#Clean and process some text, including lowercase and removal of special characters
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s.,!?]', '', text)
    return text

dialogue_pairs_with_metadata['context'] = dialogue_pairs_with_metadata['context'].apply(preprocess_text)
dialogue_pairs_with_metadata['response'] = dialogue_pairs_with_metadata['response'].apply(preprocess_text)


In [31]:
#Going to utilize batch preprocessing below with specialized tokens.
context_texts = dialogue_pairs_with_metadata['context'].tolist()
response_texts = dialogue_pairs_with_metadata['response'].tolist()

#Tokenize context and response
context_tokens = tokenizer(context_texts, padding=True, truncation=True, max_length=50, return_tensors='pt')
response_tokens = tokenizer(response_texts, padding=True, truncation=True, max_length=50, return_tensors='pt')

#Add the tokenizations to dataframe
dialogue_pairs_with_metadata['context_tokens'] = context_tokens['input_ids'].tolist()
dialogue_pairs_with_metadata['response_tokens'] = response_tokens['input_ids'].tolist()
print(dialogue_pairs_with_metadata[['context', 'context_tokens', 'response', 'response_tokens']].head())


                                             context  \
0  then why did you go see mr. koehler in the fir...   
1                                           hi, joe.   
2                    frank  what are you doing here?   
3  i need to ask you a question.  what made you g...   
4                   you know i cant talk about that.   

                                      context_tokens  \
0  [8524, 1521, 750, 345, 467, 766, 285, 81, 13, ...   
1  [5303, 11, 2525, 68, 13, 50257, 50257, 50257, ...   
2  [8310, 962, 220, 644, 389, 345, 1804, 994, 30,...   
3  [72, 761, 284, 1265, 345, 257, 1808, 13, 220, ...   
4  [5832, 760, 1312, 18548, 1561, 546, 326, 13, 5...   

                                            response  \
0                              because he called me.   
1                    frank  what are you doing here?   
2  i need to ask you a question.  what made you g...   
3                   you know i cant talk about that.   
4  im not asking for names or specifics.  i ju

In [32]:
#Model loading
model = GPT2LMHeadModel.from_pretrained('distilgpt2')
model.resize_token_embeddings(len(tokenizer))

#TRaining according to our dataset
class GPTDialogueDataset(Dataset):
    def __init__(self, df):
        self.context_tokens = df['context_tokens'].tolist()
        self.response_tokens = df['response_tokens'].tolist()

    def __len__(self):
        return len(self.context_tokens)

    def __getitem__(self, idx):
        input_ids = torch.tensor(self.context_tokens[idx])
        labels = torch.tensor(self.response_tokens[idx])
        return {'input_ids': input_ids, 'labels': labels}

train_dataset = GPTDialogueDataset(dialogue_pairs_with_metadata)

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [33]:
#Training and testing sets, should probably split earlier in code
train_df, eval_df = train_test_split(dialogue_pairs_with_metadata, test_size=0.2, random_state=37)

#Split into training and evlauation datasets
train_dataset = GPTDialogueDataset(train_df)
eval_dataset = GPTDialogueDataset(eval_df)

In [34]:
def objective(trial):
    #hyperparameters to try
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-4)
    num_train_epochs = trial.suggest_int('num_train_epochs', 2, 5)
    batch_size = trial.suggest_categorical('batch_size', [2, 4, 8])

    training_args = TrainingArguments(
        output_dir='./gpt2_chatbot',
        learning_rate=learning_rate,
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=batch_size,
    )

    trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset)
    eval_result = trainer.evaluate()
    return eval_result['eval_loss']

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)
print("Best Hyperparameters:", study.best_params)

[I 2024-10-06 16:19:37,637] A new study created in memory with name: no-name-7cec7e2c-8015-4fa1-ba67-21dd863a4259
<ipython-input-34-55bb045795f6>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-4)


[I 2024-10-06 16:20:17,849] Trial 0 finished with value: 15.632522583007812 and parameters: {'learning_rate': 1.1380662628871282e-05, 'num_train_epochs': 2, 'batch_size': 4}. Best is trial 0 with value: 15.632522583007812.
<ipython-input-34-55bb045795f6>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-4)


[I 2024-10-06 16:20:55,789] Trial 1 finished with value: 15.632522583007812 and parameters: {'learning_rate': 3.959204387092146e-05, 'num_train_epochs': 3, 'batch_size': 8}. Best is trial 0 with value: 15.632522583007812.
<ipython-input-34-55bb045795f6>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-4)


[I 2024-10-06 16:21:33,657] Trial 2 finished with value: 15.632522583007812 and parameters: {'learning_rate': 3.443925183486824e-05, 'num_train_epochs': 3, 'batch_size': 4}. Best is trial 0 with value: 15.632522583007812.
<ipython-input-34-55bb045795f6>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-4)


[I 2024-10-06 16:22:11,496] Trial 3 finished with value: 15.632522583007812 and parameters: {'learning_rate': 1.3940918626159624e-05, 'num_train_epochs': 2, 'batch_size': 4}. Best is trial 0 with value: 15.632522583007812.
<ipython-input-34-55bb045795f6>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-4)


...

[I 2024-10-06 16:24:04,817] Trial 6 finished with value: 15.632522583007812 and parameters: {'learning_rate': 9.781656911845137e-05, 'num_train_epochs': 3, 'batch_size': 8}. Best is trial 0 with value: 15.632522583007812.
<ipython-input-34-55bb045795f6>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-4)


[I 2024-10-06 16:24:42,964] Trial 7 finished with value: 15.632522583007812 and parameters: {'learning_rate': 6.477382534448103e-05, 'num_train_epochs': 3, 'batch_size': 4}. Best is trial 0 with value: 15.632522583007812.
<ipython-input-34-55bb045795f6>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-4)


[I 2024-10-06 16:25:21,051] Trial 8 finished with value: 15.632522583007812 and parameters: {'learning_rate': 4.6894663268345854e-05, 'num_train_epochs': 3, 'batch_size': 8}. Best is trial 0 with value: 15.632522583007812.
<ipython-input-34-55bb045795f6>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-4)


[I 2024-10-06 16:25:59,023] Trial 9 finished with value: 15.632522583007812 and parameters: {'learning_rate': 9.194349216218714e-05, 'num_train_epochs': 3, 'batch_size': 2}. Best is trial 0 with value: 15.632522583007812.


Best Hyperparameters: {'learning_rate': 1.1380662628871282e-05, 'num_train_epochs': 2, 'batch_size': 4}


 - All return the same finished value with different parameters, need to investigate further

In [35]:
#Check the best hyperparameters
print("Best Hyperparameters:", study.best_params)
print("/n Best Trial:")
print(f"  Number: {study.best_trial.number}")
print(f"  Value: {study.best_trial.value}")
print(f"  Params: {study.best_trial.params}")

Best Hyperparameters: {'learning_rate': 1.1380662628871282e-05, 'num_train_epochs': 2, 'batch_size': 4}
/n Best Trial:
  Number: 0
  Value: 15.632522583007812
  Params: {'learning_rate': 1.1380662628871282e-05, 'num_train_epochs': 2, 'batch_size': 4}


In [36]:
#Define the training arguments with hyperparameters from above
training_args = TrainingArguments(
    output_dir='./gpt2_chatbot',
    learning_rate=study.best_params['learning_rate'],
    num_train_epochs=study.best_params['num_train_epochs'],
    per_device_train_batch_size=study.best_params['batch_size'],
    gradient_accumulation_steps=8,
    save_steps=2000,
    evaluation_strategy="steps",
    eval_steps=2000,
    warmup_steps=500,
    #Change to mixed precision training
    fp16=True,
    #Load in best model at end of training loss
    load_best_model_at_end=True,
    #Possible early stopping
    metric_for_best_model='eval_loss',
    #Want lowest eval loss
    greater_is_better=False
)

#Trainer with early stopping and scheduler
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss,Validation Loss
2000,2.009600,1.977448
4000,2.000600,1.970572
6000,1.987200,1.962473
8000,1.970900,1.961637
10000,1.983900,1.960658


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=11080, training_loss=2.143164110700146, metrics={'train_runtime': 1892.2449, 'train_samples_per_second': 187.388, 'train_steps_per_second': 5.855, 'total_flos': 4523699994624000.0, 'train_loss': 2.143164110700146, 'epoch': 1.999864630101753})

 - On A100, Took 1 hour, costing about 15 credits.

##Evaluations

In [37]:
#Calculate metrics before evaluation
def calculate_metrics(predictions, labels, tokenizer):
    #Flatten token list
    preds_flat = [token for pred in predictions for token in pred]
    labels_flat = [token for label in labels for token in label]
    #Filter out padding tokens
    pad_token_id = tokenizer.pad_token_id
    preds_flat = [token for token in preds_flat if token != pad_token_id]
    labels_flat = [token for token in labels_flat if token != pad_token_id]
    #Calculate accuracy, recall, and F1 score
    accuracy = accuracy_score(labels_flat, preds_flat)
    recall = recall_score(labels_flat, preds_flat, average='weighted')
    f1 = f1_score(labels_flat, preds_flat, average='weighted')
    return accuracy, recall, f1

In [38]:
#Evaluation of model function
def evaluate_model(dataset, model, tokenizer):
    #Need to move model to GPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    model.eval()
    predictions = []
    labels = []

    for sample in dataset:
        #Need to place input tensors to same location as model
        input_ids = sample['input_ids'].unsqueeze(0).to(device)
        label_ids = sample['labels'].tolist()

        #Attention mask for input id's, avoid issue
        attention_mask = (input_ids != tokenizer.pad_token_id).long()
        with torch.no_grad():
            #Output with attention mask
            output = model.generate(
                input_ids,
                attention_mask=attention_mask,
                max_length=50,
                num_beams=2,
                early_stopping=True,
                pad_token_id=tokenizer.pad_token_id
            )
            predictions.append(output[0].tolist())
            labels.append(label_ids)
    accuracy, recall, f1 = calculate_metrics(predictions, labels, tokenizer)
    print(f"Accuracy: {accuracy}, Recall: {recall}, F1 Score: {f1}")

evaluate_model(train_dataset, model, tokenizer)

ValueError: Found input variables with inconsistent numbers of samples: [2401822, 2347497]

 - Should include batching in above for time speed up

In [ ]:
def multi_turn_conversation(prompt, model, tokenizer, max_turns=5, max_length=50):
    conversation_history = f"{tokenizer.bos_token} {prompt}"
    print(f"User: {prompt}")

    for _ in range(max_turns):
        input_ids = tokenizer.encode(conversation_history, return_tensors='pt')
        with torch.no_grad():
            output = model.generate(input_ids, max_length=max_length, num_beams=2, no_repeat_ngram_size=2, early_stopping=True)
        response = tokenizer.decode(output[0], skip_special_tokens=True).strip()

        print(f"Bot: {response}")

        user_input = input("User: ")
        conversation_history += f" {tokenizer.sep_token} {user_input}"

prompt = "Do you think you are lucky? Do ya punk?"
multi_turn_conversation(prompt, model, tokenizer)

**SAVE MODEL CONFIG AFTER TRAINING**

In [ ]:
#Save the model and tokenizer in a file
save_path = './trained_model_gpt2'
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

 - When done, download model by going to "File" in top left bar, and selecting "Download" on file.

*Do Not Run Below Just Yet*

In [ ]:
'''
#Call in model for future use
model = GPT2LMHeadModel.from_pretrained(save_path)
tokenizer = GPT2Tokenizer.from_pretrained(save_path)
'''

 - General setup for FastAPI, but will check later

In [ ]:
'''
app = FastAPI()

@app.post("/generate")
async def generate_response(request: dict):
    prompt = request['prompt']
    response = generate_response_beam_search(prompt)
    return {"response": response}

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)
'''